In [ ]:
import sys
sys.path.append('..')
import pandas as pd
import numpy as np
import myfunction as mf
path_data_raw = "C:/Users/dell/OneDrive/file/"
path_country_nc = "C:/Users/dell/OneDrive/file/nc"
path_one_spdb = 'C:/Users/dell/OneDrive/file/SPDB/'
drive_letter = 'E:'

path_pre = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/pretreatment/"
path_match = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/match/"
path_semdata = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/"
path_temp = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/temp/"
path_split_lr = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/lr/"
path_split_sw = drive_letter + "/wyy/code_project/running_outcome/final_data/SPDB/part0_treat/semdata/sw/"
path_var_data = drive_letter + "/wyy/SPDB_database/data/raw/"

mark_num = "25"
meta_name = "meta_data.csv"
list_color = ["#ee877c", "#8bd0e3", "#6abeae", "#808eaf", "#f7bba8", "#acb4cc", "#b5e0d5", "#e86462", "#a89687"]

In [ ]:
# Function Preparation
# 函数准备
from factor_analyzer import FactorAnalyzer
from openpyxl import Workbook
from openpyxl.styles import PatternFill
from openpyxl.utils.dataframe import dataframe_to_rows
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
def preprocess(df_data, treat_method, list_var=None):
    """
    sem uses raw uniformly and then uses this function to preprocess it. 
    normalisation: normalize the maximum and minimum values.
    standardisation:Z-score normalization.
    box_cox:box_cox transformation
    others: normal-z-score non-normal-logarithmic
    list_var is left out to process all columns, otherwise only the element columns are processed.
    ---
    sem统一使用raw 然后再使用这个函数去预处理 
    normalization:最大值最小值归一化
    standardization:Z-score标准化
    box_cox:box_cox变换
    其他：正态-z-score  非正-对数
    list_var不填就是对所有列处理 否则只对元素列进行处理
    """
    
    if list_var is None:
        list_var = df_data.columns
    
    if treat_method == "normalization":
        df_data[list_var] = (df_data[list_var] - df_data[list_var].min()) / (df_data[list_var].max() - df_data[list_var].min())
    elif treat_method == "standardization":
        df_data[list_var] = (df_data[list_var] - df_data[list_var].mean()) / df_data[list_var].std()
    elif treat_method == "box_cox":
        numeric_cols = df_data[list_var].select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            df_data[col], _ = stats.boxcox(df_data[col])
    elif treat_method == "log":
        df_data[list_var] = np.log10(df_data[list_var])
    elif treat_method == "yeo_johnson":
        numeric_cols = df_data[list_var].select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            df_data[col], _ = stats.yeojohnson(df_data[col])
    elif treat_method == "auto":
        df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")
        list_yj_meta = df_meta['var_name'][df_meta['var_type0']==1]
        list_bc_meta = df_meta['var_name'][df_meta['var_type0']==0]
        list_var = [item for item in list_var if 'log_' not in item]
        list_yj = list(set(list_yj_meta) & set(list_var))
        list_bc = list(set(list_bc_meta) & set(list_var))
        print('YJ:',list_yj)
        print('BC:',list_bc)

        for col in list_yj:
            try:
                df_data[col], _ = stats.yeojohnson(df_data[col])
            except:
                print('error:',col)
                pass
        for col in list_bc:
            try:
                df_data[col], _ = stats.boxcox(df_data[col])
            except:
                print('error:',col)
                pass
    else:
        # 没指定方法就默认先检查数据是否正态分布，正态跳过，非正态直接yeojohnson
        # No method is specified, the default first check whether the data is normally distributed, 
        # normal skip, non-normal directly yeojohnson
        for var in list_var:
            p_val = stats.normaltest(df_data[var]).pvalue
            if p_val > 0.05:
                print('normal:',var)
                pass
            else:
                try:
                    df_data[var], _ = stats.yeojohnson(df_data[var])
                except:
                    print('error:',var)
                    pass
                
    return df_data

def count_variance(df_sem_data):
    """
    Calculate all variables
    ---
    计算所有变量
    """
    print(df_sem_data.shape)
    mean = df_sem_data.mean()
    std_dev = df_sem_data.std()
    variance = df_sem_data.var()
    df_results = pd.DataFrame({
        'var': df_sem_data.columns,
        'mean': mean.values,
        'SD': std_dev.values,
        'variance': variance.values
    })

    min_variance = df_results["variance"].min()
    max_variance = df_results["variance"].max()
    print(min_variance, max_variance)
    print(max_variance/min_variance)
    return df_results

def convert_temp(df_o, from_scale, to_scale):
    """"
    Convert units of temperature variables, 
    C:Celsius, F:Fahrenheit, K:Kelvin, 
    now only supports variables with global coverage, 
    non-global zeros are also converted.
    ---
    转换温度变量的单位,
    C:Celsius,F:Fahrenheit,K:Kelvin,
    现在只支持覆盖全球的变量,
    非全球的0也会被变换
    """
    df = df_o.copy()
    df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")
    list_all_var = df_meta['var_name'][(df_meta['var_select' + mark_num]==1)&(df_meta['var_temp']==1)].to_list()
    for col in list_all_var:
        if from_scale == "C":
            if to_scale == "F":
                df[col] = (df[col] * 9/5) + 32
            elif to_scale == "K":
                df[col] = df[col] + 273.15
        elif from_scale == "F":
            if to_scale == "C":
                df[col] = (df[col] - 32) * 5/9
            elif to_scale == "K":
                df[col] = ((df[col] - 32) * 5/9) + 273.15
        elif from_scale == "K":
            if to_scale == "C":
                df[col] = df[col] - 273.15
            elif to_scale == "F":
                df[col] = ((df[col] - 273.15) * 9/5) + 32
    return df

def encode_categorical(df_train, list_train):
    """
    Dummy coding of categorical variables
    ---
    对分类变量进行虚拟编码
    """
    df_encoded = pd.get_dummies(df_train[list_train], drop_first=True)
    list_var = df_encoded.columns.tolist()
    df_train = df_train.drop(columns=list_train)
    df_train = pd.concat([df_train, df_encoded], axis=1)
    return df_train, list_var

def get_var_str(df, list_train):
    """
    Categorical variables are sequentially coded, 
    here inversely coded as text, 
    to facilitate virtual coding.
    ---
    分类变量被顺序编码  
    这里逆编码为文本 
    目的是为了方便虚拟编码
    """
    for var_change in list_train:
        var_df = pd.read_csv('E:/wyy/SPDB_database/data/spdb/' +var_change +'.csv')
        var_df = var_df.drop(columns=["50%"])
        df = df.merge(var_df, left_on=var_change, right_on='id', how='left')
        print(df.columns)
        df = df.drop(columns=[var_change + '_x'])
        df = df.rename(columns={var_change+'_y': var_change})
        df = df.drop(columns=['id'])
    return df


## GEO-DATA

In [ ]:
# Data conversion of spatial geographic variables
# 对空间地理变量进行数据转换
df_data_raw = pd.read_csv(path_match + 'geo_global.csv')

df_data_raw = df_data_raw[(df_data_raw['lon_grid'] != -180) & (df_data_raw['lat_grid'] != -90)]
df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")
df_data_raw = convert_temp(df_data_raw, "K", "C")

list_meta = df_meta["var_name"][df_meta["var_select" + mark_num]==1].to_list()
list_data = df_data_raw.columns.tolist()
list_select = list(set(list_meta)&set(list_data))

df_data1 = df_data_raw[list_select]
second_treat = "auto"

if second_treat == "N":
    pass
else:
    df_data1 = preprocess(df_data1, treat_method=second_treat)
feature_type = "standardization"
df_data1 = preprocess(df_data1, treat_method=feature_type)
df_sem_variance = count_variance(df_data1)

# ------------------------------------------------------------------
treat_mark1 = "au_"
treat_mark2 = "to_"

df_sem_variance.to_csv(path_temp + "variance_geo"+mark_num+".csv", index=False)


df_data = pd.concat([df_data1, df_data_raw[["lon_grid", "lat_grid", 'year']]], axis=1)
sem_data_name = "geo_global_"+ treat_mark1 + treat_mark2 + mark_num+".csv"
print(sem_data_name)
print(df_data.columns)
df_data.to_csv(path_semdata + sem_data_name, index=False)

## FISH

In [ ]:
# Data conversion of fish data
# 对鱼类数据进行数据转换

data_type = "fish"
df_data_raw = pd.read_csv(path_match + 'lr_' + data_type +'_avg.csv')
df_data_raw = df_data_raw[(df_data_raw['length_last'].notna())&(df_data_raw['weight_last'].notna())&(df_data_raw['troph_last'].notna())]
df_data_raw = df_data_raw.rename(columns={"length_last": "sp_length", 'weight_last': 'sp_weight', 'troph_last': 'sp_troph'})

df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")
list_meta = df_meta["var_name"][df_meta["var_select" + mark_num]==1].to_list()
# ------------------------------------------------------------------
df_features = df_data_raw.drop(columns="value")
list_categorical = ["organ",'habitat']
for var in list_categorical:
    df_features = df_data_raw.drop(columns=var)
list_fea = df_features.columns.tolist()
list_select = list(set(list_fea)&set(list_meta))
df_data1 = df_features[list_select]

list_categorical_t = ["po_chain"]
for cat in list_categorical_t:
    if "po_chain" in df_data1.columns:
        df_data1 = df_data1.drop(columns="po_chain")

print(df_data1.shape)
negative_treat = "N"
second_treat = "auto"
if second_treat == "N":
    pass
else:
    df_data1 = preprocess(df_data1, treat_method=second_treat)
feature_type = "standardization"
df_data2 = preprocess(df_data1, treat_method=feature_type)

df_data2 = pd.concat([df_data2, df_data_raw["value"]], axis=1)
if second_treat == "N":
    pass
else:
    # box_cox, yeo_johnson
    value_treat = "box_cox"
    df_data2 = preprocess(df_data2, value_treat, ["value"])
value_treat2 = "standardization"
df_data2 = preprocess(df_data2, value_treat2, ["value"])
df_sem_variance = count_variance(df_data2)

df_data3 = df_data_raw[list_categorical]

print(df_data3.head())
df_data4, list_new_categorical = encode_categorical(df_data3, list_categorical)

df_data5 = df_data_raw[list_categorical_t]
# ------------------------------------------------------------------
treat_mark1 = "au_"
treat_mark2 = "to_"

df_sem_variance.to_csv(path_temp + "variance_lr"+data_type+".csv", index=False)

df_data = pd.concat([df_data2, df_data4, df_data5], axis=1)
df_data = pd.concat([df_data, df_data_raw[["lon_grid", "lat_grid", 'year']]], axis=1)
sem_data_name = "sem_lr_" +data_type+"_"+ treat_mark1 + treat_mark2 + mark_num+"_avg.csv"

colnames = df_data.columns.tolist()
habitat_columns = [col for col in colnames if 'habitat' in col]
print(habitat_columns)
print(df_data.shape)
if len(habitat_columns) == 1:
    df_data = df_data.rename(columns={habitat_columns[0]: 'habitat'})

if second_treat == "auto":
    df_geo = pd.read_csv(path_semdata + "geo_global_"+ treat_mark1 + treat_mark2 + mark_num+".csv")
elif second_treat == "N" and feature_type == "N":
    df_geo = pd.read_csv(path_match + "geo_all.csv")
merged_df = pd.merge(df_geo, df_data, on=['lat_grid', 'lon_grid', 'year'],how='right')

merged_df.to_csv(path_semdata + sem_data_name, index=False)
print(merged_df.shape)
print(sem_data_name)

## WATER

In [ ]:
# Data conversion of water data
# 对水体数据进行数据转换

sw_mark = 's7'
df_data_raw = pd.read_csv(path_match + 'sw_' + sw_mark + '_avg.csv')
print("raw data shape",df_data_raw.shape)
df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")
df_features = df_data_raw.drop(columns="value")

print("feature data shape",df_features.shape)

list_sp_var = ["sp_length", 'sp_weight', 'sp_troph']
list_var_select = df_meta["var_name"][df_meta["var_select" + mark_num]==1].to_list()
list_var_select = [var for var in list_var_select if var not in list_sp_var]
list_fea = df_data_raw.columns.tolist()
list_select = list(set(list_fea)&set(list_var_select))
df_data1 = df_features[list_select]

list_categorical_t = ["po_chain"]
for cat in list_categorical_t:
    if "po_chain" in df_data1.columns:
        df_data1 = df_data1.drop(columns="po_chain")

negative_treat = "N"

second_treat = "auto"

if second_treat == "N":
    pass
else:
    df_data1 = preprocess(df_data1, treat_method=second_treat)
print("data1 shape",df_data1.shape)

feature_type = "standardization"

if feature_type == "N":
    df_data2 = pd.concat([df_data1, df_data_raw["value"]], axis=1)
else:
    df_data2 = preprocess(df_data1, treat_method=feature_type)
    df_data2 = pd.concat([df_data2, df_data_raw["value"]], axis=1)
# ------------------------------------------------------------------
if second_treat == "N":
    pass
else:
    value_treat = "box_cox"
    df_data2 = preprocess(df_data2, value_treat, ["value"])
if feature_type == "N":
    pass
else:
    df_data2 = preprocess(df_data2, feature_type, ["value"])
    df_sem_variance = count_variance(df_data2)
    print("data2 shape",df_data2.shape)
# ------------------------------------------------------------------

df_data5 = df_data_raw[list_categorical_t]

print("data5 shape",df_data5.shape)

if second_treat == "auto":
    treat_mark1 = "au_"
    treat_mark2 = "to_"
elif second_treat == "N" and feature_type == "N":
    treat_mark1 = "raw_"
    treat_mark2 = "raw_"
elif second_treat == "N" and feature_type == "standardization":
    treat_mark1 = "raw_"
    treat_mark2 = "std_"

df_data = pd.concat([df_data2, df_data5], axis=1)
df_data = pd.concat([df_data, df_data_raw[["lon_grid", "lat_grid", 'year']]], axis=1)

sem_data_name = "sem_sw_" + sw_mark + "_" + treat_mark1 + treat_mark2 + mark_num+"_avg.csv"
df_sem_variance.to_csv(path_temp +"variance_sw_" + sw_mark + ".csv", index=False)

if second_treat == "auto":
    df_geo = pd.read_csv(path_semdata + "geo_global_"+ treat_mark1 + treat_mark2 + mark_num+".csv")
elif second_treat == "N" and feature_type == "N":
    df_geo = pd.read_csv(path_match + "geo_all.csv")

merged_df = pd.merge(df_geo, df_data, on=['lat_grid', 'lon_grid', 'year'],how='right')

merged_df.to_csv(path_semdata + sem_data_name, index=False)

merged_df.to_csv(path_semdata + sem_data_name, index=False)
print("merge shape",merged_df.shape)
print(sem_data_name)


## WATER-FISH

In [ ]:
# Data conversion of spatial and temporal overlap data for water and fish
# 对水和鱼时空重叠数据进行转换

data_type = "fish"
df_data_raw = pd.read_csv(path_match + 'lr_sws7_' + data_type +'_avg.csv')

df_data_raw = df_data_raw[(df_data_raw['length_last'].notna())&(df_data_raw['weight_last'].notna())&(df_data_raw['troph_last'].notna())]
df_data_raw = df_data_raw.rename(columns={"length_last": "sp_length", 'weight_last': 'sp_weight', 'troph_last': 'sp_troph'})
df_meta = pd.read_csv(path_data_raw + meta_name, encoding="utf-8")

# ------------------------------------------------------------------
df_features = df_data_raw.drop(columns="value")
df_features = df_data_raw.drop(columns="sw_value")

list_categorical = ["organ",'habitat']
for var in list_categorical:
    df_features = df_data_raw.drop(columns=var)

list_fea = df_features.columns.tolist()
list_meta = df_meta["var_name"][df_meta["var_select" + mark_num]==1].to_list()
list_select = list(set(list_fea)&set(list_meta))

df_data1 = df_features[list_select]
# ------------------------------------------------------------------
list_categorical_t = ["po_chain"]
for cat in list_categorical_t:
    if "po_chain" in df_data1.columns:
        df_data1 = df_data1.drop(columns="po_chain")

print(df_data1.shape)
# ------------------------------------------------------------------

negative_treat = "N"
# ------------------------------------------------------------------
# normalization, standardization, box_cox
second_treat = "auto"

if second_treat == "N":
    pass
else:
    df_data1 = preprocess(df_data1, treat_method=second_treat)


feature_type = "standardization"
df_data2 = preprocess(df_data1, treat_method=feature_type)
# ------------------------------------------------------------------
df_data2 = pd.concat([df_data2, df_data_raw[["value",'sw_value']]], axis=1)
if second_treat == "N":
    pass
else:
    value_treat = "box_cox"
    df_data2 = preprocess(df_data2, value_treat, ["value",'sw_value'])
value_treat2 = "standardization"
df_data2 = preprocess(df_data2, value_treat2, ["value",'sw_value'])
df_sem_variance = count_variance(df_data2)
# ------------------------------------------------------------------
df_data3 = df_data_raw[list_categorical]
print(df_data3.head())
df_data4, list_new_categorical = encode_categorical(df_data3, list_categorical)
# ------------------------------------------------------------------
df_data5 = df_data_raw[list_categorical_t]

treat_mark1 = "au_"
treat_mark2 = "to_"

df_sem_variance.to_csv(path_temp + "variance_lr_sws7_"+data_type+"_avg.csv", index=False)

df_data = pd.concat([df_data2, df_data4, df_data5], axis=1)
df_data = pd.concat([df_data, df_data_raw[["lon_grid", "lat_grid", 'year']]], axis=1)

sem_data_name = "sem_lr_sws7_"+ treat_mark1 + treat_mark2 + mark_num+"_avg.csv"

colnames = df_data.columns.tolist()
habitat_columns = [col for col in colnames if 'habitat' in col]
print(habitat_columns)
print(df_data.shape)

if len(habitat_columns) == 1:
    df_data = df_data.rename(columns={habitat_columns[0]: 'habitat'})
if second_treat == "auto":
    df_geo = pd.read_csv(path_semdata + "geo_global_"+ treat_mark1 + treat_mark2 + mark_num+".csv")
elif second_treat == "N" and feature_type == "N":
    df_geo = pd.read_csv(path_match + "geo_all.csv")

merged_df = pd.merge(df_geo, df_data, on=['lat_grid', 'lon_grid', 'year'],how='right')
merged_df.to_csv(path_semdata + sem_data_name, index=False)
print(merged_df.shape)
print(sem_data_name)

# Data are grouped according to time and space
#数据根据时空进行分组
treat_mark1 = "au_"
treat_mark2 = "to_"
df = pd.read_csv(path_semdata + "sem_lr_sws7_" + treat_mark1 + treat_mark2 + mark_num+"_avg.csv")
df['cluster'] = df['lon_grid'].astype(str) + '&' + df['lat_grid'].astype(str) + '&' + df['year'].astype(str)
cluster_dict = {cluster: i for i, cluster in enumerate(df['cluster'].unique())}
df['cluster'] = df['cluster'].map(cluster_dict)
cluster_counts = df['cluster'].value_counts()
total_clusters = len(cluster_counts)

df.to_csv(path_semdata + "sem_lr_sws7_" + treat_mark1 + treat_mark2 + mark_num + "_cluster.csv", index=False)